## ¿Qué es una CNN?

Las CNNs son redes neuronales especialmente diseñadas para procesar datos con estructura de cuadrícula, como imágenes. Su arquitectura está inspirada en el sistema visual biológico y utiliza tres operaciones principales:

### 1. Convolución
Aplica filtros (kernels) que detectan características locales como bordes, texturas y patrones.

$$
(f * g)(x, y) = \sum_{i=-k}^{k} \sum_{j=-k}^{k} f(i, j) \cdot g(x-i, y-j)
$$

### 2. Pooling
Reduce la dimensionalidad espacial, conservando las características más importantes (max pooling, average pooling).

### 3. Capas Completamente Conectadas
Al final de la red, realizan la clasificación basándose en las características extraídas.

## Ventajas de las CNNs

- **Invarianza espacial**: Detectan características sin importar su posición en la imagen
- **Compartición de parámetros**: Los mismos filtros se aplican en toda la imagen
- **Jerarquía de características**: Capas iniciales detectan bordes, capas intermedias formas, capas finales objetos completos
- **Eficiencia**: Menos parámetros que redes completamente conectadas

## Implementación: CNN para Clasificación MNIST

Construiremos una CNN simple para clasificar dígitos manuscritos del dataset MNIST.

In [ ]:
# Importar bibliotecas necesarias
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Configurar dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Usando dispositivo: {device}')

### Arquitectura de la CNN

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        # Primera capa convolucional: 1 canal de entrada (escala de grises) -> 32 canales
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # 28x28 -> 14x14
        
        # Segunda capa convolucional: 32 canales -> 64 canales
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # 14x14 -> 7x7
        
        # Capas completamente conectadas
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.relu3 = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 10)  # 10 clases (dígitos 0-9)
    
    def forward(self, x):
        # Bloque convolucional 1
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        
        # Bloque convolucional 2
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        
        # Aplanar para capas densas
        x = x.view(-1, 64 * 7 * 7)
        
        # Capas completamente conectadas
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

# Crear modelo
model = SimpleCNN().to(device)
print(model)

### Preparar Dataset MNIST

In [ ]:
# Transformaciones: convertir a tensor y normalizar
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # Media y desviación estándar de MNIST
])

# Descargar y cargar datos
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

print(f'Tamaño del conjunto de entrenamiento: {len(train_dataset)}')
print(f'Tamaño del conjunto de prueba: {len(test_dataset)}')

### Visualizar Datos

In [ ]:
# Mostrar algunas imágenes de ejemplo
examples = enumerate(train_loader)
batch_idx, (example_data, example_targets) = next(examples)

fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    ax.imshow(example_data[i][0], cmap='gray')
    ax.set_title(f'Etiqueta: {example_targets[i]}')
    ax.axis('off')
plt.tight_layout()
plt.show()

### Entrenamiento

In [ ]:
# Definir función de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Función de entrenamiento
def train_epoch(model, device, train_loader, optimizer, criterion):
    model.train()
    train_loss = 0
    correct = 0
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
    
    train_loss /= len(train_loader)
    accuracy = 100. * correct / len(train_loader.dataset)
    return train_loss, accuracy

# Función de evaluación
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader)
    accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, accuracy

In [ ]:
# Entrenar el modelo
epochs = 5
train_losses, test_losses = [], []
train_accs, test_accs = [], []

for epoch in range(1, epochs + 1):
    train_loss, train_acc = train_epoch(model, device, train_loader, optimizer, criterion)
    test_loss, test_acc = test(model, device, test_loader, criterion)
    
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    train_accs.append(train_acc)
    test_accs.append(test_acc)
    
    print(f'Época {epoch}/{epochs}')
    print(f'  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
    print(f'  Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%')
    print()

### Visualizar Resultados

In [ ]:
# Gráficas de pérdida y precisión
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(range(1, epochs+1), train_losses, label='Train Loss', marker='o')
ax1.plot(range(1, epochs+1), test_losses, label='Test Loss', marker='s')
ax1.set_xlabel('Época')
ax1.set_ylabel('Pérdida')
ax1.set_title('Pérdida durante el Entrenamiento')
ax1.legend()
ax1.grid(True)

ax2.plot(range(1, epochs+1), train_accs, label='Train Accuracy', marker='o')
ax2.plot(range(1, epochs+1), test_accs, label='Test Accuracy', marker='s')
ax2.set_xlabel('Época')
ax2.set_ylabel('Precisión (%)')
ax2.set_title('Precisión durante el Entrenamiento')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

### Predicciones en Nuevas Imágenes

In [ ]:
# Hacer predicciones en un batch de prueba
model.eval()
test_examples = enumerate(test_loader)
batch_idx, (test_data, test_targets) = next(test_examples)

with torch.no_grad():
    test_data = test_data.to(device)
    output = model(test_data)
    predictions = output.argmax(dim=1, keepdim=True)

# Visualizar predicciones
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    ax.imshow(test_data[i][0].cpu(), cmap='gray')
    pred = predictions[i].item()
    true = test_targets[i].item()
    color = 'green' if pred == true else 'red'
    ax.set_title(f'Pred: {pred} | Real: {true}', color=color)
    ax.axis('off')
plt.tight_layout()
plt.show()

## Conclusiones

Esta CNN simple logra alta precisión en MNIST (~98-99%) con solo 5 épocas de entrenamiento. Las CNNs son extremadamente efectivas para tareas de visión porque:

1. Aprenden jerarquías de características automáticamente
2. Son invariantes a traslaciones
3. Requieren menos parámetros que redes densas
4. Generalizan bien a datos no vistos

**Arquitecturas modernas** como ResNet, VGG, EfficientNet extienden estos principios con técnicas avanzadas:
- Conexiones residuales (skip connections)
- Batch normalization
- Arquitecturas más profundas
- Bloques de atención